In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.stats as ss

import sklearn.model_selection as ms 

In [2]:
# load your data (don't touch, just run)
"""
data = []
f = open('covtype.data','r')
while(1):
    line = f.readline()
    if  len(line) < 100:
        print line
    
    if len(line) == 0: break
    data.append(np.array([float(k) for k in line.split(',')]))
    if len(data) % 100000 == 0:
        print len(data)
        
f.close
data = np.vstack(data)
N = data.shape[0]
idx = np.random.permutation(N)


X_test = data[:N/5,:]
X_train = data[N/5:,:]
y_test = X_test[:,-1]
y_train = X_train[:,-1]
X_test = X_test[:,:-1]
X_train = X_train[:,:-1]


sio.savemat('covtype.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})

data = sio.loadmat('covtype.mat')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train'][0]
y_test = data['y_test'][0]

y_idx_train = [np.where(np.equal(y_train,k))[0] for k in np.unique(y_train)]

for i in xrange(len(y_idx_train)):
    y_idx = y_idx_train[i]
    y_idx_train[i] = y_idx[np.random.choice(len(y_idx),len(y_idx)/1000+1,replace=False)]
    
y_idx_train = np.hstack(y_idx_train)
y_idx_train = np.random.permutation(y_idx_train)

X_train = X_train[y_idx_train,:]
y_train = y_train[y_idx_train]

sio.savemat('covtype_reduced.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})


"""

"\ndata = []\nf = open('covtype.data','r')\nwhile(1):\n    line = f.readline()\n    if  len(line) < 100:\n        print line\n    \n    if len(line) == 0: break\n    data.append(np.array([float(k) for k in line.split(',')]))\n    if len(data) % 100000 == 0:\n        print len(data)\n        \nf.close\ndata = np.vstack(data)\nN = data.shape[0]\nidx = np.random.permutation(N)\n\n\nX_test = data[:N/5,:]\nX_train = data[N/5:,:]\ny_test = X_test[:,-1]\ny_train = X_train[:,-1]\nX_test = X_test[:,:-1]\nX_train = X_train[:,:-1]\n\n\nsio.savemat('covtype.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})\n\ndata = sio.loadmat('covtype.mat')\nX_train = data['X_train']\nX_test = data['X_test']\ny_train = data['y_train'][0]\ny_test = data['y_test'][0]\n\ny_idx_train = [np.where(np.equal(y_train,k))[0] for k in np.unique(y_train)]\n\nfor i in xrange(len(y_idx_train)):\n    y_idx = y_idx_train[i]\n    y_idx_train[i] = y_idx[np.random.choice(len(y_idx),len(y_idx)/1000+1,repla

In [3]:

data = sio.loadmat('../covertype_release/covtype_reduced.mat')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train'][0]
y_test = data['y_test'][0]

print (np.unique(y_train), np.unique(y_test))

print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)


[1. 2. 3. 4. 5. 6. 7.] [1. 2. 3. 4. 5. 6. 7.]
(468, 54) (116202, 54) (468,) (116202,)


In [4]:
def entropy(label):
    entropy = 0
    for i in np.unique(label):
        count_i = len(label[label == i])
        count = len(label)
        entropy += -(count_i/count)*np.log2((count_i/count)) if count_i != count else 0
    return entropy

def cond_entropy(label,split):
    cond_entropy = 0
    for i in np.unique(label):
        for j in np.unique(split):
            count_i_j = len(label[(label == i) & (split == j)])
            count_j = len(split[split == j])
            count = len(split)
            cond_entropy += -(count_i_j/count)*np.log2(count_i_j/count_j) if count_i_j != 0 else 0

    return cond_entropy

random_sequences = sio.loadmat('../covertype_release/random_sequences.mat')

s1 = random_sequences['s1'][0]
s2 = random_sequences['s2'][0]

print ('entropy = ', entropy(s1))
print ('conditional entropy = ', cond_entropy(s1,s2))

entropy =  3.314182323161083
conditional entropy =  3.3029598816135173


In [5]:
def find_best_split(x,y):
    best_feat = 0
    splitval = 0 #sorted(np.unique(X_train[:, best_feat]))[1] #0
    max_info_gain = -np.inf
    entropy_val = entropy(y)
    for feat in range(x.shape[1]):        
        for temp_splitval in sorted(np.unique(x[:, feat])):
            y_split = y*0
            y_split[x[:, feat] < temp_splitval] = 1
            y_split[x[:, feat] >= temp_splitval] = 0
            if entropy_val - cond_entropy(y, y_split) > max_info_gain:
                best_feat = feat
                splitval = temp_splitval
                max_info_gain = entropy_val - cond_entropy(y, y_split)

    set1 = np.argwhere(x[:, best_feat] < splitval).reshape(-1, )
    set2 = np.argwhere(x[:, best_feat] >= splitval).reshape(-1, )

    return best_feat, splitval, set1, set2

best_feat, splitval, set1, set2 = find_best_split(X_train, y_train)
y_new = y_train * 0
y_new[set1] = 1
print ('information gained in first step', entropy(y_train) - cond_entropy(y_train,y_new))

information gained in first step 0.34265604135138417


In [6]:
def purity(y):
    return ss.mode(y)[1]/len(y+0.)
    
class Node:
    def __init__(self,  sample_idx, nodeid,  is_leaf = True):
        self.is_leaf = is_leaf
        self.id = nodeid
        self.sample_idx = sample_idx
        self.children = []
        
    def visit_node(self, x):
        if self.is_leaf:
            return self.label
        elif x[self.splitfeat] < self.splitval:
            return self.children[0].visit_node(x)
        return self.children[1].visit_node(x)
        
    def add_split_details(self, splitfeat, splitval)  :
        self.splitfeat = splitfeat
        self.splitval = splitval
    
        
class Tree:
    def __init__(self, x,y):
        m = len(y)
        self.x = x
        self.y = y
        self.maxid = -1
        self.root = self.construct_node(np.array(range(m)))
        self.leaves = [self.root]
        
    def print_tree(self):
        print ('printing tree...')
        def print_node(parent, node):
            print (node.id, )
            
            if parent is not None:
                print (', parent ', parent.id,)
            else:
                print (', ROOT', )
                
            print (', label ', node.label, )
            if node.is_leaf: 
                print (', LEAF, ', 'nsamples %d, purity %.2f' %(len(node.sample_idx), purity(self.y[node.sample_idx])))
                #node_ys = self.y[node.sample_idx]
                #vals, counts = np.unique(node_ys, return_counts=True)
                #if len(vals[counts == np.max(counts)]) > 1:
                #    print (f"CONFLICT HERE in {len(vals[counts == np.max(counts)])}")
            else:
                print (', NONLEAF, split %d, val %.2f' % (node.splitfeat, node.splitval))
            if not node.is_leaf:
                for ch in node.children:
                    print_node(node, ch)
        print_node(None, self.root)
        
        
    def construct_node(self, sample_idx):
        node = Node(sample_idx, self.maxid + 1,  True)
        node_ys = self.y[node.sample_idx]
        vals, counts = np.unique(node_ys, return_counts=True)
        if len(vals[counts == np.max(counts)]) == 1:
            node.label = vals[counts == np.max(counts)][0] #ss.mode(self.y[sample_idx])[0].item() #0 # fill me in 
        else:
            conflict_y_counts = [(i, len(self.y[self.y==i])) for i in vals[counts == np.max(counts)] ]
            conflict_y_counts = sorted(conflict_y_counts, key=lambda a: a[1])
            node.label = conflict_y_counts[-1][0]
        node.entropy = entropy(self.y[sample_idx])
        node.num_mistakes = np.sum(np.not_equal(node.label, self.y[sample_idx]))
        self.maxid += 1
        return node
        
    def report_train_err(self):
        total_mistakes = 0
        for leaf in self.leaves:
            total_mistakes += leaf.num_mistakes
        return total_mistakes / (len(self.y)+0.)
        
    def predict(self,x):
        return self.root.visit_node(x)
    

In [7]:
def get_test_err(tree):
    # get test error
    num_test_mistakes = 0
    for k in range(len(y_test)):
        x,y = X_test[k,:],y_test[k]
        if y != tree.predict(x):
            num_test_mistakes += 1
    return num_test_mistakes / (len(y_test)+0.)

tree = Tree(X_train,y_train)
tree.print_tree()
print ('current train err:', tree.report_train_err())
print ('current test err:', get_test_err(tree))


# my first split
best_feat, splitval, set1, set2 = find_best_split(X_train, y_train)
 
left_child = tree.construct_node(set1)
right_child = tree.construct_node(set2)
tree.root.is_leaf = False
tree.leaves.pop(tree.leaves.index(tree.root))
tree.root.add_split_details(splitfeat = best_feat, splitval = splitval)


tree.root.children = [left_child, right_child]
tree.leaves.extend(tree.root.children)

tree.print_tree()
print ('one step train err:', tree.report_train_err())
print ('one step test err:', get_test_err(tree))

while (tree.maxid <=51):
    curr_node = tree.leaves[0] #tree.leaves.pop(0)
    if purity(tree.y[curr_node.sample_idx]).item() == 1: #overfitting param
        print (f"Node {curr_node.id} is pure")
        tree.leaves.pop(tree.leaves.index(curr_node))
        continue
    #print (f"Node {curr_node.id} purity = {np.around(purity(tree.y[curr_node.sample_idx]), 3)}, ")
    print(f"test error after {tree.maxid//2} steps: {np.around(get_test_err(tree), 3)}, train error: {np.around(tree.report_train_err(), 3)}")

    best_feat, splitval, set1, set2 = find_best_split(tree.x[curr_node.sample_idx], tree.y[curr_node.sample_idx])
    left_child = tree.construct_node(np.array(set1))
    right_child = tree.construct_node(np.array(set2))
    tree.leaves[0].is_leaf = False
    tree.leaves[0].add_split_details(splitfeat = best_feat, splitval = splitval)
    tree.leaves[0].children = [left_child, right_child]
    tree.leaves.extend(tree.leaves[0].children)
    tree.leaves.pop(0)

print ('25 step train err:', tree.report_train_err())
print ('25 step test err:', get_test_err(tree))


printing tree...
0
, ROOT
, label  2.0
, LEAF,  nsamples 468, purity 0.44
current train err: 0.5641025641025641
current test err: 0.3138069912738163
printing tree...
0
, ROOT
, label  2.0
, NONLEAF, split 0, val 2844.00
1
, parent  0
, label  2.0
, LEAF,  nsamples 116, purity 0.46
2
, parent  0
, label  1.0
, LEAF,  nsamples 352, purity 0.51
one step train err: 0.5021367521367521
one step test err: 0.4959983477048588
test error after 1 steps: 0.496, train error: 0.502
test error after 2 steps: 0.776, train error: 0.491
test error after 3 steps: 0.818, train error: 0.549
test error after 4 steps: 0.826, train error: 0.551
test error after 5 steps: 0.546, train error: 0.543
test error after 6 steps: 0.365, train error: 0.524
test error after 7 steps: 0.322, train error: 0.526
test error after 8 steps: 0.324, train error: 0.524
test error after 9 steps: 0.316, train error: 0.524
test error after 10 steps: 0.596, train error: 0.517
test error after 11 steps: 0.596, train error: 0.513
test 

In [8]:
tree.print_tree()

printing tree...
0
, ROOT
, label  2.0
, NONLEAF, split 0, val 2844.00
1
, parent  0
, label  2.0
, NONLEAF, split 0, val 2525.00
3
, parent  1
, label  2.0
, NONLEAF, split 9, val 757.00
7
, parent  3
, label  2.0
, NONLEAF, split 1, val 299.00
15
, parent  7
, label  1.0
, NONLEAF, split 0, val 2739.00
31
, parent  15
, label  2.0
, LEAF,  nsamples 3, purity 0.33
32
, parent  15
, label  1.0
, LEAF,  nsamples 3, purity 0.33
16
, parent  7
, label  1.0
, NONLEAF, split 0, val 3172.00
33
, parent  16
, label  5.0
, LEAF,  nsamples 1, purity 1.00
34
, parent  16
, label  1.0
, LEAF,  nsamples 2, purity 0.50
8
, parent  3
, label  1.0
, NONLEAF, split 0, val 2965.00
17
, parent  8
, label  2.0
, NONLEAF, split 2, val 23.00
35
, parent  17
, label  2.0
, LEAF,  nsamples 4, purity 0.25
36
, parent  17
, label  1.0
, LEAF,  nsamples 4, purity 0.50
18
, parent  8
, label  1.0
, NONLEAF, split 0, val 3172.00
37
, parent  18
, label  1.0
, LEAF,  nsamples 10, purity 0.70
38
, parent  18
, labe